In [159]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt# Matlab-style plotting
from sodapy import Socrata


In [160]:
client = Socrata("data.cityofnewyork.us", None)
results = client.get_all("8vv7-7wx3")

# Convert to pandas DataFrame
emc_df = pd.DataFrame.from_records(results)
emc_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16827 entries, 0 to 16826
Data columns (total 5 columns):
date_and_time         16827 non-null object
email_body            16304 non-null object
notification_title    16827 non-null object
notificationtype      16827 non-null object
record_id             16827 non-null object
dtypes: object(5)
memory usage: 657.4+ KB


In [161]:
emc_df.drop(columns = ['email_body'], axis = 1, inplace=True)
emc_df.duplicated(keep = "first")
emc_df['notificationtype'] = emc_df['notificationtype'].astype('category')
emc_df['date_and_time'] = pd.to_datetime(emc_df['date_and_time'])
emc_df['date_and_time'] = emc_df['date_and_time'].dt.date

In [162]:
emc_df.loc[:,'notificationtype'].value_counts()

Road Closure                  5148
Mass Transit Disruption       2366
Missing Adult                 1352
Utility                       1320
Weather                       1238
Public Awareness              1081
Environmental                  901
Fire                           771
Mass Transit Restoration       722
Aerial (Fly-Over)              562
Public Health                  453
Drills / Exercises             316
School Notification            207
Missing Child / Adolescent     180
Parking                        127
Structural                      67
Infrastructure                  16
Name: notificationtype, dtype: int64

In [163]:
emc_df.loc[:,'notification_title'].value_counts()

[blank]                                                                        1728
CSO Advisory (CW)                                                               403
CSO Advisory - (Citywide)                                                       131
CSO Advisories (CW)                                                              56
Intermittent Wind Gusts (CW)                                                     34
CSO Advisory                                                                     30
Air Quality Health Advisory (CW)                                                 27
Traffic Delays - FDR Drive (MN)                                                  19
MTA Suspension - A Train (QN)                                                    19
Power Outage (SI)                                                                16
Power Outage - Middle Village (QN)                                               16
MTA Restoration - A Train (QN)                                              

In [131]:
t_df["notification_title"]=t_df["notification_title"].apply(lambda x: np.NaN if x== "[blank]" else x)
t_df["notification_title"].count()

15099

In [153]:
print(emc_df['date_and_time'].min())
print(emc_df['date_and_time'].max())

1900-01-01
2020-04-26


In [154]:
nyc_case_hosp = pd.read_csv('https://raw.githubusercontent.com/nychealth/coronavirus-data/master/case-hosp-death.csv')
nyc_case_hosp.drop(columns= ['HOSPITALIZED_CASE_COUNT', 'DEATH_COUNT'],inplace=True)
nyc_case_hosp.rename(columns= {'DATE_OF_INTEREST':'Date', 'NEW_COVID_CASE_COUNT':'Newcase'}, inplace = True)
nyc_case_hosp['Date'] = pd.to_datetime(nyc_case_hosp['Date'])
nyc_case_hosp['Date'] = nyc_case_hosp['Date'].dt.date
nyc_case_hosp.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 54 entries, 0 to 53
Data columns (total 2 columns):
Date       54 non-null object
Newcase    54 non-null int64
dtypes: int64(1), object(1)
memory usage: 944.0+ bytes


In [155]:
print(nyc_case_hosp['Date'].min())
print(nyc_case_hosp['Date'].max())

2020-03-03
2020-04-25


In [156]:
import datetime
import time
str='2020-03-02'
d=datetime.date(*map(int,str.split('-')))

emc_df = emc_df[ emc_df['date_and_time'].apply(lambda x: x > d)]
emc_df.rename(columns ={'date_and_time': 'Date'}, inplace = True )

In [157]:
emc_type_count_df = emc_df.groupby(['Date','notificationtype'])['record_id'].count().reset_index(name = 'count')
emc_type_count_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 257 entries, 0 to 256
Data columns (total 3 columns):
Date                257 non-null object
notificationtype    257 non-null category
count               257 non-null int64
dtypes: category(1), int64(1), object(1)
memory usage: 5.1+ KB


In [158]:
merge_df = pd.merge(nyc_case_hosp, emc_type_count_df, on ='Date')
print(merge_df)

           Date  Newcase          notificationtype  count
0    2020-03-03        2         Aerial (Fly-Over)      1
1    2020-03-03        2   Mass Transit Disruption      1
2    2020-03-03        2  Mass Transit Restoration      1
3    2020-03-03        2              Road Closure      5
4    2020-03-03        2                   Weather      1
5    2020-03-04        5             Environmental      1
6    2020-03-04        5   Mass Transit Disruption      4
7    2020-03-04        5  Mass Transit Restoration      3
8    2020-03-04        5             Missing Adult      1
9    2020-03-04        5             Public Health      1
10   2020-03-04        5              Road Closure      3
11   2020-03-04        5                   Utility      1
12   2020-03-04        5                   Weather      2
13   2020-03-05        3             Public Health      1
14   2020-03-05        3              Road Closure      6
15   2020-03-06        7   Mass Transit Disruption      2
16   2020-03-0